<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5 Data Class Builders
 - `collections.namedtuple`
 - `typing.NamedTuple`
 - `@dataclasses.dataclass`

Note. `typing.TypedDict` does not build concrete classes that you can instantiate.

In [ ]:
class Coordinate:
  def __init__(self, lat, lon):
    self.lat = lat
    self.lon = lon

In [ ]:
moscow = Coordinate(55.76, 37.62)
moscow

In [ ]:
location = Coordinate(55.76, 37.62)
location == moscow

False

The data class builders provide the necessary `__init__`, `__repr__` and `__eq__` methods automatically.

In [ ]:
from collections import namedtuple

Coordinate = namedtuple('Coordinate', 'lat lon')
issubclass(Coordinate, tuple)

True

In [ ]:
moscow = Coordinate(55.756, 37.617)
moscow # Useful __repr__

Coordinate(lat=55.756, lon=37.617)

In [ ]:
moscow == Coordinate(lat=55.756, lon=37.617) # meaningful __eq__

True

Newer NamedTuple provides the same functionality, adding a type annotation to each field

In [ ]:
import typing

Coordinate = typing.NamedTuple('Coordinate', [('lat', float), ('lon', float)])

In [ ]:
issubclass(Coordinate, tuple)

True

In [ ]:
typing.get_type_hints(Coordinate)

{'lat': float, 'lon': float}

In [ ]:
Coordinate = typing.NamedTuple('Coordinate', lat=float, lon=float)

**Warning**
Although `NamedTuple` appears in the `class` statement as a superclass, it's actually not. It uses the advanced functionality of a metaclass to customize the creation of the user's class.

In [ ]:
from typing import NamedTuple

class Coordinate(NamedTuple):
  lat: float
  lon: float

  def __str__(self):
    ns = 'N' if self.lat >= 0 else 'S'
    we = 'E' if self.lon >= 0 else 'W'
    return f'{abs(self.lat):1.f}°{ns}, {abs(self.lon):1.f}°{we}'

In [ ]:
issubclass(Coordinate, typing.NamedTuple)

TypeError: issubclass() arg 2 must be a class, a tuple of classes, or a union

In [ ]:
issubclass(Coordinate, tuple)

True

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
  lat: float
  lon: float = 10

  def __str__(self):
    ns = 'N' if self.lat >= 0 else 'S'
    we = 'E' if self.lon >= 0 else 'W'
    return f'{abs(self.lat):1.f}°{ns}, {abs(self.lon):1.f}°{we}'

The difference is in the `class` statement itslef. The `@dataclass` decorator does not depend on inheritance or a metaclass.

### Mutable instances

By default, `@dataclass` produces mutable classes. But the decorator accepts a keyword argument `frozen`. When `frozen=True`, the class will raise an exception if you try to assign a value to a field after the instance is initialized.

### Class statement syntax
Only `typing.NamedTuple` and `dataclass` support the regular `class` statement syntax.

### Construct dict

In [ ]:
import dataclasses

dataclasses.asdict(Coordinate(37.28, 127.0))

{'lat': 37.28, 'lon': 127.0}

### Get field names and default values

In [ ]:
[f.default for f in dataclasses.fields(Coordinate(37.28, 127.0))]

[<dataclasses._MISSING_TYPE at 0x7fdfb9293dc0>, 10]

In [ ]:
typing.get_type_hints(Coordinate)

{'lat': float, 'lon': float}

In [ ]:
# namedtuple._replace returns a new instance with some attribute values repalced
moscow._replace(lon=38)

Coordinate(lat=55.756, lon=37.617)

In [ ]:
suwon = Coordinate(37.28, 127.0)
dataclasses.replace(suwon, lat=37.29)

Coordinate(lat=37.29, lon=127.0)

## Classic Named Tuples

In [ ]:
from collections import namedtuple
# arguments: classname and iterables of string (or a single space-delimited string)
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', '36.933', (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population='36.933', coordinates=(35.689722, 139.691667))

In [ ]:
tokyo.population

'36.933'

In [ ]:
tokyo[1]

'JP'

A named tuple offers a few attributes and methods in addition to those inherited from tuples.

Ex. `_fields` class attribute, `_make(iterable)` class method, `_asdict()` instance method

In [ ]:
City._fields

('name', 'country', 'population', 'coordinates')

In [ ]:
Coordinate = namedtuple('Coordinate', 'lat lon')

In [ ]:
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))

In [ ]:
delhi = City._make(delhi_data) # _make buildes City from an iterable

In [ ]:
delhi._asdict()

{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': Coordinate(lat=28.613889, lon=77.208889)}

`._asdict()` is useful to serialize the data in JSON format

In [ ]:
import json

json.dumps(delhi._asdict())

'{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}'

Since Python 3.7, `namedtuple` accepts the `defaults` keyword-only argument providing an iterable of N default values for each of the N rightmost fields of the class.

In [ ]:
Coordinate = namedtuple('Coordinate', 'lat lon reference', defaults=['WGS84'])
Coordinate(0, 0)

Coordinate(lat=0, lon=0, reference='WGS84')

In [ ]:
Coordinate._field_defaults

{'reference': 'WGS84'}

#### Hacking a namedtuple to inject a method

In [ ]:
Card = namedtuple('Card', ['rank', 'suit'])

In [ ]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [ ]:
Card.suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
def spades_high(card):
  rank_value = FrenchDeck.ranks.index(card.rank)
  suit_value = card.suit_values[card.suit]
  return rank_value * len(card.suit_values) + suit_value

Card.overall_rank = spades_high

In [ ]:
lowest_card = Card('2', 'clubs')
highest_card = Card('A', 'spades')

In [ ]:
lowest_card.overall_rank()

0

In [ ]:
highest_card.overall_rank()

51

## Typed Named Tuples

In [ ]:
from typing import NamedTuple

# every instnace must be annotated with a type
class Coordinate(NamedTuple):
  lat: float
  lon: float
  reference: str = 'WGS84'

### No Runtime Effect

In [ ]:
trash = Coordinate('Ni!', None)

In [ ]:
print(trash)

Coordinate(lat='Ni!', lon=None, reference='WGS84')


In [ ]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 23.0 MB/s eta 0:00:00


In [ ]:
!mypy test.py

test.py:9: error: Argument 1 to "Coordinate" has incompatible type "str"; expected "float"  [arg-type]
test.py:9: error: Argument 2 to "Coordinate" has incompatible type "None"; expected "float"  [arg-type]
Found 2 errors in 1 file (checked 1 source file)


## Meaning of Variable Annotations

At import time--when a module is loaded--Python does read them to build the `__annotations__` dictionary that `typing.NamedTuple` and `@dataclass` then use to enhance the class.

In [ ]:
class DemoPlainClass:
  a: int
  b: float = 1.1
  c = 'spam'

In [ ]:
from demo_plain import DemoPlainClass

DemoPlainClass.__annotations__

{'a': int, 'b': float}

In [ ]:
DemoPlainClass.a

AttributeError: type object 'DemoPlainClass' has no attribute 'a'

In [ ]:
DemoPlainClass.b

1.1

In [ ]:
DemoPlainClass.c

'spam'

In [ ]:
import typing

class DemoNTClass(typing.NamedTuple):
  a: int
  b: float = 1.1
  c = 'spam'

In [ ]:
from demo_nt import DemoNTClass

DemoNTClass.__annotations__

{'a': int, 'b': float}

`a` and `b` class attributes are descriptors. For now, think of them as similar to property getters: methods that don't require teh explicit call operator `()` to retrieve an instance attribute.

In [ ]:
DemoNTClass.a

_tuplegetter(0, 'Alias for field number 0')

In [ ]:
DemoNTClass.b

_tuplegetter(1, 'Alias for field number 1')

In [ ]:
DemoNTClass.c

'spam'

In [ ]:
DemoNTClass.__doc__

'DemoNTClass(a, b)'

In [ ]:
nt = DemoNTClass(8)
nt.a

8

In [ ]:
nt.b

1.1

In [ ]:
nt.c

'spam'

In [ ]:
nt.a = 'hi'

AttributeError: can't set attribute

In [ ]:
from dataclasses import dataclass

@dataclass
class DemoDataClass:
  a: int
  b: float = 1.1
  c = 'spam'

In [ ]:
from demo_dc import DemoDataClass

DemoDataClass.__annotations__

{'a': int, 'b': float}

In [ ]:
DemoDataClass.__doc__

'DemoDataClass(a: int, b: float = 1.1)'

In [ ]:
DemoDataClass.a

AttributeError: type object 'DemoDataClass' has no attribute 'a'

In [ ]:
DemoDataClass.b

1.1

In [ ]:
DemoDataClass.c

'spam'

In [ ]:
dc = DemoDataClass(9)

In [ ]:
dc.a

9

In [ ]:
dc.b

1.1

In [ ]:
dc.c

'spam'

As mentioned, `DemoDataClass` instances are mutable--and no type checking is done at runtime.

In [ ]:
dc.a = 10

In [ ]:
dc.b = 'oops'

In [ ]:
dc.c = 'whatever'

In [ ]:
dc.d = 'secret_stash'

In [ ]:
dc

DemoDataClass(a=10, b='oops')

## More about `@dataclass`

Its signature:
`@dataclass(*, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)`

Mutable default values are a common source of bugs for beginning Python developers.

In [ ]:
from club_wrong import ClubMember

In [ ]:
from dataclasses import dataclass, field

@dataclass
class ClubMember:
  name: str
  guests: list[str] = field(default_factory=list)
  athelete: bool = field(default=False, repr=False)

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from club_generic import ClubMember

ClubMember("George", ['a', 'b', 'c'], False)

ClubMember(name='George', guests=['a', 'b', 'c'])

In [ ]:
!pip install mypy

In [ ]:
!mypy hackerclub.py

Success: no issues found in 1 source file


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!python resource.py

Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd Edition',
    creators = datetime.date(2018, 11, 19),
    date = <ResourceType.BOOK: 1>,
    type = 'improving teh design of existing code',
    description = 'EN',
    language = ['computer programming', 'OOP'],
    subjects = [],
)


## Pattern Matching Class Instances
 1. simple
 2. keyword
 3. positional

### Keyword Class Patterns

In [30]:
import typing

class City(typing.NamedTuple):
  continent: str
  name: str
  country: str

cities = [
    City('Asia', 'Tokyo', 'JP'),
    City('Asia', 'Delhi', 'IN'),
    City('North America', 'Mexico City', 'MX'),
    City('North America', 'New York', 'US'),
    City('South America', 'Sao Paulo', 'BR'),
]

In [33]:
def match_asian_cities():
  results = []
  for city in cities:
    match city:
      case City(continent='Asia', country=cc):
        results.append(cc)
  return results

In [34]:
match_asian_cities()

['JP', 'IN']

### Positional Class Patterns

In [35]:
def match_asian_cities_pos():
  results = []
  for city in cities:
    match city:
      case City('Asia'):
        results.append(city)
  return results

In [36]:
match_asian_cities_pos()

[City(continent='Asia', name='Tokyo', country='JP'),
 City(continent='Asia', name='Delhi', country='IN')]

In [37]:
def match_asian_countries_pos():
  results = []
  for city in cities:
    match city:
      case City('Asia', _, country):
        results.append(country)
  return results

As you can see, `__match_args` declares the names of the attributes in the order they will be used in positional patterns

In [38]:
City.__match_args__

('continent', 'name', 'country')